In [ ]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import random
import os

from supervised.automl import AutoML

# 한글 폰트 설정
import platform
plt.rc('font', family='Malgun Gothic')
# 마이너스 폰트 설정
plt.rc('axes', unicode_minus=False) 
# 글씨 선명하게 출력하는 설정
%config InlineBackend.figure_format = 'retina'

""" # Warning 제거
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) 
warnings.simplefilter(action='ignore', category=UserWarning)  
pd.set_option('mode.chained_assignment',  None)   
#pd.set_option('mode.chained_assignment', 'warn') """

In [ ]:
train = pd.read_csv(r'data\train.csv')
test = pd.read_csv(r'data\test.csv')
target = pd.read_csv(r'data\sample_submission.csv')

In [ ]:
train

In [ ]:
train_corr = train.drop(['propertyType','suburbName','ID'],axis=1)
corr = train_corr.corr()
plt.figure(figsize=(20,20))
sns.heatmap(corr, annot=True, fmt='.2f', cmap='coolwarm')

In [ ]:

corr_target = abs(corr['monthlyRent(us_dollar)'])
relevant_features = corr_target[corr_target>0.05]
pd.DataFrame(relevant_features)

In [ ]:
col = list(pd.DataFrame(relevant_features).index)
col.append('bedrooms')
propertyType = list(train.propertyType.unique())


for i in col:
    for j in col:
#        for k in propertyType:
        if i==j:
            continue
        else:
            sns.scatterplot(data=train,x=i,y=j,hue='propertyType')
            plt.title(f'{i}-{j}')
            plt.show()



In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(42) # Seed 고정

In [ ]:
automl = AutoML(mode="Compete", eval_metric='mae')

In [ ]:
train_x = train.drop(['ID','monthlyRent(us_dollar)'],axis=1)
train_y = train['monthlyRent(us_dollar)']
test_x = test.drop(['ID'],axis=1)

In [ ]:
automl.fit(train_x, train_y)

In [ ]:
pred = automl.predict(test_x)

In [ ]:
target.iloc[:, 1] = pred
target

In [ ]:
target.to_csv("target.csv", index=False)